In [17]:
from scrape.request import getLinksFromSearch, createSearchLink, convertDataToDataFrame, dfPriceManipulation, loadLinksFromSearch
import requests
import re
from bs4 import BeautifulSoup
import tqdm
from scrape.PageScrape import AmazonPageScrape
import numpy as np

In [29]:
search_term = "Shatavari"
header = {
    "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36 RuxitSynthetic/1.0 v377946003586811470 t8360729428027585528 ath259cea6f altpriv cvcv=2 smf=0",
    "Upgrade-Insecure-Requests": "1",
    "Accept-Encoding": "gzip deflate br",
    "Accept": "text/html"
}
proxies = {
    "http": "",
    "https": ""
}


In [30]:
links = []
iteration = 1

session = requests.Session()
session.headers.update(header)
session.proxies.update(proxies)

while len(links) < 50:
    search_url = createSearchLink("https://www.amazon.com", f"{search_term} herbal supplement", iteration)
    search = session.get(search_url, headers=header)
    soup = BeautifulSoup(search.content, "lxml")
    new_links = getLinksFromSearch(soup, "https://www.amazon.com")
    iteration += 1
    
    if len(links) + len(new_links) >= 50:
        numLinks = 50 - len(links)
        new_links = new_links[:numLinks]
        links.extend(new_links)
    
    else:
        links.extend(new_links)
        
    print(len(links))

24
48
50


In [31]:
data = loadLinksFromSearch(links, headers=None, session=session)

URL requests: 100%|██████████| 50/50 [00:16<00:00,  3.07links/s]


In [32]:
data_scrape = [ AmazonPageScrape(data[index], data[index].url, search_term) for index in tqdm.tqdm(links)]

100%|██████████| 50/50 [00:19<00:00,  2.50it/s]


In [33]:
df = convertDataToDataFrame(data_scrape)
df = dfPriceManipulation(df)
df.head()

,source,url,title,brand,productType,asin,price,rank,rating,numRatings,formFactor,firstAvailable,uses,ingredients,origin,pricePerCount,pricePerOz
0,Amazon,https://www.amazon.com/Organic-India-Shatavari...,Organic India Shatavari Herbal Supplement - Su...,ORGANIC INDIA,Shatavari,B003PGAOYS,$18.99,11667.0,4.5,1569.0,"Capsules,Capsule","June 4, 2010",None,"Organic Shatavari Root, Organic Vegetable Pull...",None,$0.21,NaN
1,Amazon,https://www.amazon.com/Organic-Essential-Femal...,Organic India Essential Female Herbal Suppleme...,ORGANIC INDIA,Shatavari,B08KRKP391,$20.99,68518.0,4.3,147.0,Capsule,"October 5, 2020",Reproductive Health,"Organic Shatavari, Organic Guduchi, Organic Va...",None,$0.23,NaN
2,Amazon,https://www.amazon.com/Shatavari-Ashwagandha-S...,Cona Nature Organic Shatavari Powder Capsules ...,Cona Nature,Shatavari,B089K93764,$15.98,76333.0,4.5,271.0,Capsule,"June 2, 2020",None,Shatavari Capsules 2250mg(3 capsules per servi...,India,$0.13,NaN
3,Amazon,https://www.amazon.com/Shatavari-Powder-Aspara...,Shatavari Powder (Asparagus Racemosus) | For B...,Bixa BOTANICAL,Shatavari,B01M1KA2QZ,$15.99,544540.0,4.1,13.0,Powder,"November 14, 2016",None,Shatawari Root,None,NaN,$2.28
4,Amazon,https://www.amazon.com/Organic-Shatavari-Botan...,Jiva Botanicals - Shatavari Capsules 1200 mg -...,Jiva LLC,Shatavari,B07K69PWK8,$19.99,117373.0,4.3,398.0,None,"August 26, 2019",None,- Organic Shatavari Powder (Asparagus Racemosu...,None,$0.33,NaN


In [34]:
df.tail()

,source,url,title,brand,productType,asin,price,rank,rating,numRatings,formFactor,firstAvailable,uses,ingredients,origin,pricePerCount,pricePerOz
45,Amazon,https://www.amazon.com/Organic-Ashwagandha-130...,"Organic Ashwagandha 2,100 mg - 100 Vegan Capsu...",NaturaLife Labs,Shatavari,B06ZYHJYD5,$15.95,167.0,4.5,38827.0,"Powders, Capsule","April 18, 2017","Immune Support, Stress Relief, Thyroid Support",None,None,$0.16,NaN
46,Amazon,https://www.amazon.com/Ashwagandha-Terrestris-...,"Ashwagandha with Tribulus Terrestris, Shatavar...",Ireya Ayurveda,Shatavari,B09SWWGYTS,$14.99,207864.0,5.0,1.0,Capsule,"February 19, 2022",None,"Ashwagandha Powder, Tribulus Terrestris Root, ...",India,$0.17,NaN
47,Amazon,https://www.amazon.com/Herbal-Shatavari-Powder...,Herbal Hills Shatavari Powder (Asparagus racem...,Isha Agro Developers Pvt. Ltd.,Shatavari,B0BFDZ5DQ6,$16.99,850097.0,NaN,NaN,None,"October 22, 2020",None,Shatavari Powder,None,NaN,$1.06
48,Amazon,https://www.amazon.com/Biome-Clinically-Valida...,For The Biome Stress Therapy | Clinically Prov...,For The Biome,Shatavari,B08XC65BC2,$22.39,171380.0,3.9,29.0,Compostable Infusion Bag,"March 18, 2021",None,"Organic German Chamomile, Passionflower, Calen...",USA,$1.60,NaN
49,Amazon,https://www.amazon.com/Uplift-Shatavari-Veggie...,Uplift Shatavari Organic Veggie Capsules-120 C...,Uplift Live Life Elevated!,Shatavari,B08NB96R1S,$14.99,490285.0,5.0,2.0,"Capsule,Powder","November 12, 2020",None,Organic Shatavari Powder,None,$0.12,NaN


In [35]:
C_map = {
    "source": "A",
    "url": "B",
    "title": "C",
    "brand": "D",
    "price": "E",
    "pricePerCount": "F",
    "pricePerOz": "G",
    "asin": "Q",
    "rank": "R",
    "rating": "S",
    "numRatings": "U",
    "formFactor": "W",
    "firstAvailable": "J",
    "uses": "AC",
    "ingredients": "I",
    "origin": "P"
}

In [1]:
import gspread

In [2]:
gc = gspread.service_account()
sheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1BfdhC30XyM3rx9-wnMfnT-3vJsiL4KizN5Rk718lBS8/edit#gid=591202730")
worksheet = sheet.worksheet("Authentic Shilajit")

In [38]:
start_row = 2
total_length = len(df)
columns = df.columns

for column in tqdm.tqdm(columns):
    if column in C_map:
        sheet_col = C_map[column]
        data_col = df[column]
        
        if column in ("rank", "numRatings"):
            data_col = data_col.astype("Int64")
            data_col = data_col.fillna(-1)
            data_col = data_col.astype("string")
            data_col = data_col.replace("-1", "")
            # data_col = data_col.to_string()
        else:
            data_col = data_col.fillna("").astype("string")
            
        data_col = data_col.to_numpy().reshape(-1, 1)
        data_col = data_col.tolist()
        
        update_range = f"{sheet_col}{start_row}:{sheet_col}{start_row+total_length}"
        
        worksheet.update(update_range, data_col, value_input_option="USER_ENTERED")
        # print(column)
        # print(update_range)
        # break
        # print(len(data))
        

100%|██████████| 17/17 [00:10<00:00,  1.59it/s]
